In [1]:
import gym
from gym import spaces
from gym.utils.env_checker import check_env
import matplotlib.pyplot as plt
import numpy as np
from bucket_env import plot_stats
%matplotlib inline

pygame 2.1.2 (SDL 2.0.18, Python 3.10.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import scipy.stats as stat

In [7]:
class HanoiEnv(gym.Env):

    def __init__(self, disks=3) -> None:
        self.disks = disks
        self.turns = 0
        self.action_space = spaces.Discrete(pow(3,self.disks))
        self.observation_space = spaces.Discrete(3)
    
    def step(self,action):
        pass

    def reset(self):
        self.turns = 0
        return self.turns
    
    def render(self, mode='human'):
        pass

    def close(self):
        pass



In [8]:
env = HanoiEnv()
check_env(env)

AttributeError: 'HanoiEnv' object has no attribute 'disks'